In [ ]:
def process_dicom_files(files):
    if not files:
        return

    for widget in content_frame.winfo_children():
        widget.destroy()

    temp_cases = {}
    existing_keys = [
        (d["Name"], d["Date"].date(), d["PatientID"], d.get("Accession", ""))
        for d in all_data
    ]

    dap_accumulator = {}  # لتجميع DAP لكل حالة X-RAY

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            modality = getattr(ds, "Modality", "").upper()
            study_desc = getattr(ds, "StudyDescription", "").lower()
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            k = 0.015
            msv = 0

            # --- CT معالجة
            if modality == "CT":
                if dlp > 0:
                    if "head" in study_desc or "brain" in study_desc:
                        k = 0.0021
                    elif "neck" in study_desc:
                        k = 0.0059
                    elif "chest" in study_desc:
                        k = 0.014
                    elif "abdomen" in study_desc or "pelvis" in study_desc:
                        k = 0.015
                    msv = dlp * k
                else:
                    length = float(getattr(ds, "TotalCollimationWidth", 0))
                    dlp = ctdi * length
                    msv = dlp * k

            # --- X-RAY / CR / DR معالجة DAP
            elif modality in ["CR", "DR", "DX", "XRAY", "X-RAY"]:
                dap = 0

                # محاولة قراءة DAP من التاجات
                for tag in [(0x0018, 0x1405), (0x0018, 0x9323)]:
                    try:
                        val = float(ds.get(tag).value)
                        dap += val
                    except:
                        continue

                # نحفظ كل DAP في dict مؤقت للمريض عشان نجمّعهم
                patient_key = (name, date_obj.date(), getattr(ds, "PatientID", ""), getattr(ds, "AccessionNumber", ""))
                if patient_key not in dap_accumulator:
                    dap_accumulator[patient_key] = 0
                dap_accumulator[patient_key] += dap

                # تحديد k حسب المنطقة
                if "head" in study_desc or "brain" in study_desc:
                    k = 0.0031
                elif "chest" in study_desc:
                    k = 0.017
                elif "abdomen" in study_desc or "pelvis" in study_desc:
                    k = 0.019
                else:
                    k = 0.015

                dlp = dap_accumulator[patient_key]
                msv = dlp * k

            matched_key = None
            for existing in existing_keys:
                if is_same_person(name, existing[0]) and existing[1] == date_obj.date():
                    matched_key = existing
                    break
            key = matched_key if matched_key else (
                name, date_obj.date(), getattr(ds, "PatientID", ""), getattr(ds, "AccessionNumber", "")
            )

            img = None
            if 'PixelData' in ds:
                arr = ds.pixel_array
                if arr.max() > 0:
                    arr = (arr / arr.max() * 255).astype('uint8')
                else:
                    arr = arr.astype('uint8')
                img_pil = Image.fromarray(arr)
                if img_pil.mode not in ["L", "RGB"]:
                    img_pil = img_pil.convert("L")
                img_pil.thumbnail((400, 400))
                img = ImageTk.PhotoImage(img_pil)

            if key not in temp_cases:
                data_dict = {
                    "Name": name,
                    "Date": date_obj,
                    "CTDIvol": ctdi,
                    "DLP": dlp,
                    "mSv": msv,
                    "kFactor": k,
                    "Sex": getattr(ds, "PatientSex", ""),
                    "DOB": getattr(ds, "PatientBirthDate", ""),
                    "PatientID": getattr(ds, "PatientID", ""),
                    "Accession": getattr(ds, "AccessionNumber", ""),
                    "Images": [],
                    "Path": path,
                    "Modality": modality,
                    "Dataset": ds
                }
                temp_cases[key] = data_dict

            if img is not None:
                temp_cases[key]["Images"].append(img)

            hl7_msg = convert_to_hl7_from_table(temp_cases[key])
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}_{temp_cases[key]['PatientID']}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    all_data.extend(temp_cases.values())

    # حساب AccumulatedDose و DosePerYear
    patient_records = {}
    for data in all_data:
        pid = data["PatientID"]
        dose = data["mSv"]
        date = data["Date"]
        if pid not in patient_records:
            patient_records[pid] = []
        patient_records[pid].append((date, dose))

    accumulated_dose_dict = {}
    for pid, records in patient_records.items():
        records.sort()
        total = 0
        for date, dose in records:
            total += dose
            accumulated_dose_dict[(pid, date)] = total

    dose_per_year_dict = {}
    for pid, records in patient_records.items():
        records.sort()
        for current_date, _ in records:
            year_start = current_date - timedelta(days=364)
            total_year = sum(dose for date, dose in records if year_start <= date <= current_date)
            dose_per_year_dict[(pid, current_date)] = total_year

    for data in all_data:
        pid = data["PatientID"]
        dt = data["Date"]
        data["AccumulatedDose"] = accumulated_dose_dict.get((pid, dt), 0)
        data["DosePerYear"] = dose_per_year_dict.get((pid, dt), 0)

    display_text_data()

